In [3]:
import numpy as np
import geopy
import pandas as pd
import math
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine()
import glob

In [4]:
PATH = '/Users/richardryu/Desktop/Cantwo/w210/data/*.csv'

In [9]:
# 7/19 multiple zipcodes
def csv_par10(df, delta):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]
        result = search.by_coordinates(df.loc[i, 'lat'], df.loc[i, 'long'], radius=4)
#         try:
#             gotdata = dlist[1]
#         except IndexError:
#             gotdata = 'null
        df.loc[i, 'zipcode'] = result[0].zipcode
        df.loc[i, 'zipcode2'] = result[1].zipcode
        try:
            df.loc[i, 'zipcode3'] = result[2].zipcode
        except IndexError:
            df.loc[i, 'zipcode3'] = np.nan
        try:
            df.loc[i, 'zipcode4'] = result[3].zipcode
        except IndexError:
            df.loc[i, 'zipcode4'] = np.nan
        try:
            df.loc[i, 'zipcode5'] = result[5].zipcode
        except IndexError:
            df.loc[i, 'zipcode5'] = np.nan
        if df.loc[i, 'xcenter'] <= 0.333333:
            df.loc[i, 'detect_loc'] = 'L'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] + (delta * math.cos((math.pi / 180)*(float(df.loc[i, 'angle'])-90)))
            df.loc[i, 'new_long'] = df.loc[i, 'long'] + (delta * math.sin((math.pi / 180)*(float(df.loc[i, 'angle'])-90)))
        elif df.loc[i, 'xcenter'] > 0.666666:
            df.loc[i, 'detect_loc'] = 'R'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] - (delta * math.cos((math.pi / 180)*(float(df.loc[i, 'angle']) -90)))
            df.loc[i, 'new_long'] = df.loc[i, 'long'] - (delta * math.sin((math.pi / 180)*(float(df.loc[i, 'angle']) -90)))
        else:
            df.loc[i, 'detect_loc'] = 'C'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat']
            df.loc[i, 'new_long'] = df.loc[i, 'long']
    return df

In [5]:
pm = pd.read_csv('parking_meters.csv')
# class 0- lamp, class 1- sign-h, class 2- fh, class 3-nopark, class 4-stop, class 5- pm
pm_ff2 = pd.DataFrame(columns=['lat', 'long', 'zipcode', 'zipcode2', 'zipcode3', 'zipcode4', 'zipcode5','class'])
for i in range(len(pm)):
    result = search.by_coordinates(pm.loc[i, 'LATITUDE'], pm.loc[i, 'LONGITUDE'], radius=2)
    pm_ff2.loc[i, 'lat'] = pm.loc[i, 'LATITUDE']
    pm_ff2.loc[i, 'long'] = pm.loc[i, 'LONGITUDE']
    pm_ff2.loc[i, 'zipcode'] = result[0].zipcode
    pm_ff2.loc[i, 'zipcode2'] = result[1].zipcode
    pm_ff2.loc[i, 'class'] = 5

In [10]:
# transforming lat/long on each csv file and adding new lat/long with 2 zipcodes
conv_temp = []
for i in temp:
    conv_temp.append(csv_par10(i, 0.0001))

In [20]:
final_df12 = pd.concat([conv_temp[0], conv_temp[1], conv_temp[2], conv_temp[3], conv_temp[4], conv_temp[5], conv_temp[6], conv_temp[7], conv_temp[8], conv_temp[9], conv_temp[10], conv_temp[11], conv_temp[12], conv_temp[13], conv_temp[14]] , ignore_index=True)

In [22]:
# dropping "className" which is filled with NaN and is a duplicate of 'class'
new_final_df12 = final_df12.drop(columns = ["className"])
filt_fin_df12 = new_final_df12[new_final_df12["confidence"] > 0.5]
mosaic8 = pd.concat([filt_fin_df12, pm_ff2], ignore_index=True)

In [60]:
# 7/19 save df by zipcode in csv
def zip_df3(df):
    zip1 = list(df['zipcode'].unique())
    zip2 = list(df['zipcode2'].unique())
    zip3 = list(df['zipcode3'].unique())
    zip4 = list(df['zipcode4'].unique())
    zip5 = list(df['zipcode5'].unique())
    all_zip = list(set(zip1 + zip2 + zip3 + zip4 + zip5))
    new = []
    for i in testing_zip:
        if type(i) == str:
            new.append(i)
    for i in new:
        temp_df = df[df["zipcode"] == i]
        temp_df2 = df[df["zipcode2"] == i]
        temp_df3 = df[df["zipcode3"] == i]
        temp_df4 = df[df["zipcode4"] == i]
        temp_df5 = df[df["zipcode5"] == i]
        temp_final = pd.concat([temp_df, temp_df2, temp_df3, temp_df4, temp_df5])
        temp_final.drop_duplicates()
        temp_final.to_csv(i + '.csv', encoding = 'utf-8', index = False)

In [61]:
zip_df3(mosaic8)

In [ ]:
# old 2 zipcode stacking

In [3]:
# 7/8 save df by zipcode in csv
def zip_df2(df):
    zip1 = list(df['zipcode'].unique())
    zip2 = list(df['zipcode2'].unique())
    all_zip = list(set(zip1 + zip2))
    for i in all_zip:
        temp_df = df[df["zipcode"] == i]
        temp_df2 = df[df["zipcode2"] == i]
        temp_final = pd.concat([temp_df, temp_df2])
        temp_final.drop_duplicates()
        temp_final.to_csv(i + '.csv', encoding = 'utf-8', index = False)

In [4]:
#7/12 modified function
def csv_par9(df, delta):
    for i in range(len(df)):
        df.loc[i, 'lat'] = float(df.loc[i, 'imgname'].split('-')[0])
        df.loc[i, 'long'] = float(df.loc[i, 'imgname'].split('-')[2]) * -1.0
        df.loc[i, 'angle'] = df.loc[i, 'imgname'].split('-')[3][:-4]
        result = search.by_coordinates(df.loc[i, 'lat'], df.loc[i, 'long'], radius=3)
        df.loc[i, 'zipcode'] = result[0].zipcode
        df.loc[i, 'zipcode2'] = result[1].zipcode
        if df.loc[i, 'xcenter'] <= 0.333333:
            df.loc[i, 'detect_loc'] = 'L'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] + (delta * math.cos((math.pi / 180)*(float(df.loc[i, 'angle'])-90)))
            df.loc[i, 'new_long'] = df.loc[i, 'long'] + (delta * math.sin((math.pi / 180)*(float(df.loc[i, 'angle'])-90)))
        elif df.loc[i, 'xcenter'] > 0.666666:
            df.loc[i, 'detect_loc'] = 'R'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat'] - (delta * math.cos((math.pi / 180)*(float(df.loc[i, 'angle']) -90)))
            df.loc[i, 'new_long'] = df.loc[i, 'long'] - (delta * math.sin((math.pi / 180)*(float(df.loc[i, 'angle']) -90)))
        else:
            df.loc[i, 'detect_loc'] = 'C'
            df.loc[i, 'new_lat'] = df.loc[i, 'lat']
            df.loc[i, 'new_long'] = df.loc[i, 'long']
    return df

In [5]:
pm = pd.read_csv('parking_meters.csv')
# class 0- lamp, class 1- sign-h, class 2- fh, class 3-nopark, class 4-stop, class 5- pm
pm_ff = pd.DataFrame(columns=['lat', 'long', 'zipcode', 'zipcode2', 'class'])
for i in range(len(pm)):
    result = search.by_coordinates(pm.loc[i, 'LATITUDE'], pm.loc[i, 'LONGITUDE'], radius=2)
    pm_ff.loc[i, 'lat'] = pm.loc[i, 'LATITUDE']
    pm_ff.loc[i, 'long'] = pm.loc[i, 'LONGITUDE']
    pm_ff.loc[i, 'zipcode'] = result[0].zipcode
    pm_ff.loc[i, 'zipcode2'] = result[1].zipcode
    pm_ff.loc[i, 'class'] = 5

In [6]:
# grabbing all csv files and reading them as separate pd dataframes
temp = []
for fname in glob.glob(PATH):
    temp.append(pd.read_csv(fname))

In [7]:
# transforming lat/long on each csv file and adding new lat/long with 2 zipcodes
conv_temp = []
for i in temp:
    conv_temp.append(csv_par9(i, 0.0001))

In [8]:
final_df11 = pd.concat([conv_temp[0], conv_temp[1], conv_temp[2], conv_temp[3], conv_temp[4], conv_temp[5], conv_temp[6], conv_temp[7], conv_temp[8], conv_temp[9], conv_temp[10], conv_temp[11], conv_temp[12], conv_temp[13], conv_temp[14]] , ignore_index=True)

In [24]:
# dropping "className" which is filled with NaN and is a duplicate of 'class'
new_final_df11 = final_df11.drop(columns = ["className"])
filt_fin_df11 = new_final_df11[new_final_df11["confidence"] > 0.5]
mosaic7 = pd.concat([filt_fin_df11, pm_ff], ignore_index=True)

In [25]:
zip_df2(mosaic7)

In [26]:
mosaic7

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,zipcode,zipcode2,detect_loc,new_lat,new_long
0,39.67933178--105.0459518-270.jpg,39.6793,-105.046,270,2,0.669755,0.701563,0.637500,0.021875,0.037500,80236,80219,R,39.679432,-105.045952
1,39.66952458--104.91756090000001-270.jpg,39.6695,-104.918,270,2,0.736712,0.731250,0.685156,0.028125,0.042188,80222,80224,R,39.669625,-104.917561
2,39.67047403--105.0358949-270.jpg,39.6705,-105.036,270,0,0.854703,0.298438,0.503906,0.106250,0.392188,80236,80110,L,39.670374,-105.035895
3,39.71416001--105.0457517-270.jpg,39.7142,-105.046,270,0,0.814633,0.407031,0.521875,0.073438,0.265625,80219,80214,C,39.714160,-105.045752
4,39.71572973--105.02016189999999-270.jpg,39.7157,-105.02,270,4,0.743508,0.813281,0.583594,0.039062,0.051562,80219,80204,R,39.715830,-105.020162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38357,NaN,39.7431,-104.994,NaN,5,NaN,NaN,NaN,NaN,NaN,80264,80202,NaN,NaN,NaN
38358,NaN,39.7515,-105,NaN,5,NaN,NaN,NaN,NaN,NaN,80202,80264,NaN,NaN,NaN
38359,NaN,39.7393,-105.007,NaN,5,NaN,NaN,NaN,NaN,NaN,80204,80290,NaN,NaN,NaN
38360,NaN,39.7317,-104.935,NaN,5,NaN,NaN,NaN,NaN,NaN,80206,80220,NaN,NaN,NaN
